In [3]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [4]:
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')

In [6]:
data.columns = data.columns.str.replace(' ','')

In [7]:
data["datetime"] = data["DATE"] + ' ' + data["TIME"]

In [8]:
data.datetime = pd.to_datetime(data.datetime)

In [9]:
data.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00


In [10]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS
data['dTOTAL'] = data.dENTRIES+ data.dEXITS

In [11]:
list(data.columns.values)

['C/A',
 'UNIT',
 'SCP',
 'STATION',
 'LINENAME',
 'DIVISION',
 'DATE',
 'TIME',
 'DESC',
 'ENTRIES',
 'EXITS',
 'datetime',
 'dENTRIES',
 'dEXITS',
 'dTOTAL']

In [12]:
data.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime,dENTRIES,dEXITS,dTOTAL
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00,0.0,5.0,5.0
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00,0.0,0.0,0.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00,0.0,0.0,0.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00,0.0,0.0,0.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00,0.0,0.0,0.0


In [13]:
mask = (data.TIME == '00:00:00') | (data.TIME=='01:00:00')

In [14]:
datatest = data.ix[mask, :]

In [15]:
mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)

In [16]:
datatest2 = datatest.ix [mask2, :]

In [17]:
datatest2.shape

(28269, 15)

In [18]:
final = datatest2.groupby([datatest2.DATE,datatest2.STATION]).sum()

In [19]:
mask3 = (datatest2.dEXITS >= 0) & (datatest2.dEXITS <=5000)

In [20]:
final.shape

(2440, 5)

In [21]:
df = final.add_suffix('_Count').reset_index()
#Changing to data frame

In [146]:
df.head()

,DATE,STATION,ENTRIES_Count,EXITS_Count,dENTRIES_Count,dEXITS_Count,dTOTAL_Count
0,03/26/2016,1 AV,1409009188,1462205971,1848.0,1050.0,2898.0
1,03/26/2016,103 ST,152097386,594017455,288.0,380.0,668.0
2,03/26/2016,103 ST-CORONA,62520085,59867349,238.0,1266.0,1504.0
3,03/26/2016,104 ST,16816556,15095950,36.0,114.0,150.0
4,03/26/2016,110 ST,26187023,26772943,280.0,682.0,962.0


In [22]:
df.sort(['dTOTAL_Count'], ascending=False, inplace=True)
#sorting by descending order

/Users/anthonykim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [23]:
df.head(10)

,DATE,STATION,ENTRIES_Count,EXITS_Count,dENTRIES_Count,dEXITS_Count,dTOTAL_Count
326,03/26/2016,TIMES SQ-42 ST,3825096876,3080925551,7205.0,1990.0,9195.0
674,03/27/2016,TIMES SQ-42 ST,3825167957,3080996667,6887.0,1849.0,8736.0
56,03/26/2016,34 ST-HERALD SQ,1597992066,1303043366,3667.0,1458.0,5125.0
14,03/26/2016,14 ST-UNION SQ,1415380109,500640412,3640.0,1143.0,4783.0
2418,04/01/2016,TIMES SQ-42 ST,3825616648,3081404086,3681.0,1086.0,4767.0
58,03/26/2016,34 ST-PENN STA,2508669961,852711655,2851.0,1774.0,4625.0
405,03/27/2016,34 ST-HERALD SQ,1598065541,1303108564,3092.0,1357.0,4449.0
358,03/27/2016,125 ST,643870177,523483047,1935.0,2409.0,4344.0
9,03/26/2016,125 ST,643817979,523438192,1878.0,2429.0,4307.0
407,03/27/2016,34 ST-PENN STA,2508733787,852780381,2549.0,1725.0,4274.0


In [24]:
df['Weekend'] = 0
df['Weekend'][df['DATE'] == ('03/26/2016')] = 1
df['Weekend'][df['DATE'] == ('03/27/2016')] = 1
#Creating new Weekend column.  For Sat Mid - 4 AM and Sun Mid - 4 AM

/Users/anthonykim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/anthonykim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [25]:
df.head(30)

,DATE,STATION,ENTRIES_Count,EXITS_Count,dENTRIES_Count,dEXITS_Count,dTOTAL_Count,Weekend
326,03/26/2016,TIMES SQ-42 ST,3825096876,3080925551,7205.0,1990.0,9195.0,1
674,03/27/2016,TIMES SQ-42 ST,3825167957,3080996667,6887.0,1849.0,8736.0,1
56,03/26/2016,34 ST-HERALD SQ,1597992066,1303043366,3667.0,1458.0,5125.0,1
14,03/26/2016,14 ST-UNION SQ,1415380109,500640412,3640.0,1143.0,4783.0,1
2418,04/01/2016,TIMES SQ-42 ST,3825616648,3081404086,3681.0,1086.0,4767.0,0
58,03/26/2016,34 ST-PENN STA,2508669961,852711655,2851.0,1774.0,4625.0,1
405,03/27/2016,34 ST-HERALD SQ,1598065541,1303108564,3092.0,1357.0,4449.0,1
358,03/27/2016,125 ST,643870177,523483047,1935.0,2409.0,4344.0,1
9,03/26/2016,125 ST,643817979,523438192,1878.0,2429.0,4307.0,1
407,03/27/2016,34 ST-PENN STA,2508733787,852780381,2549.0,1725.0,4274.0,1


In [27]:
trim_df = df.head(10)

In [30]:
import numpy as np

station = trim_df['STATION']
values = trim_df['dTOTAL_Count']

plt.figure()

plt.barh(np.arange(len(station)), values)

plt.yticks(np.arange(len(station)),
           ['Station {}'.format(x) for x in station])

plt.show()